In [5]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "CS231N/assignments/assignment2/"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content

Mounted at /content/drive
/content/drive/My Drive/CS231N/assignments/assignment2/cs231n/datasets
/content


# What's this PyTorch business?

Вы написали много кода в этом задании, чтобы обеспечить целый ряд функций нейронной сети. Dropout, Batch Norm и 2D-свертки являются одними из рабочих лошадок глубокого обучения в компьютерном зрении. Вы также усердно работали над тем, чтобы сделать ваш код эффективным и векторизованным. 

Однако в последней части этого задания мы собираемся оставить вашу прекрасную кодовую базу и вместо этого перейти к одной из двух популярных сред глубокого обучения: в данном случае PyTorch (или TensorFlow, если вы решите использовать этот блокнот).

### What is PyTorch?

PyTorch — это система для выполнения динамических вычислительных графов над объектами Tensor, которые ведут себя аналогично numpy ndarray. Он поставляется с мощным механизмом автоматической дифференциации, который устраняет необходимость в ручном обратном распространении.

### Why?

* Наш код теперь будет работать на GPU! Гораздо быстрее обучение. При использовании таких фреймворков, как PyTorch или TensorFlow, вы можете использовать мощь графического процессора для своих собственных архитектур нейронных сетей без необходимости напрямую писать код CUDA (что выходит за рамки этого класса). 
* Мы хотим, чтобы вы были готовы использовать одну из этих платформ для своего проекта, чтобы вы могли экспериментировать более эффективно, чем если бы вы писали каждую функцию, которую хотите использовать, вручную. 
* Мы хотим, чтобы вы стояли на плечах гигантов! TensorFlow и PyTorch — отличные фреймворки, которые сделают вашу жизнь намного проще, и теперь, когда вы понимаете их суть, вы можете свободно их использовать :) 
* Мы хотим, чтобы вы познакомились с кодом глубокого обучения, с которым вы можете столкнуться в академии или промышленности.

### PyTorch versions
В этой записной книжке предполагается, что вы используете **PyTorch версии 1.4**. В некоторых предыдущих версиях (например, до 0.4) тензоры должны были быть заключены в объекты Variable для использования в autograd; однако переменные теперь устарели. Кроме того, версии 1.0+ отделяют тип данных тензора от его устройства и используют фабрики в стиле numpy для создания тензоров, а не напрямую вызывают конструкторы тензора.

## How will I learn PyTorch?


Justin Johnson сделал отличный [учебник](https://github.com/jcjohnson/pytorch-examples) для PyTorch. 

Вы также можете найти подробную [документацию по API](http://pytorch.org/docs/stable/index.html) здесь. Если у вас есть другие вопросы, которые не рассматриваются в документации API, [форум PyTorch](https://discuss.pytorch.org/) — гораздо лучшее место, чтобы задать их, чем StackOverflow.

## Install PyTorch 1.4 (ONLY IF YOU ARE WORKING LOCALLY)

1. Установите на свой компьютер последнюю версию Anaconda. 
2. Создайте новую среду conda, начиная с Python 3.7. В этом примере настройки мы назовем его torch_env. 
3. Запустите команду: `conda активировать torch_env` 
4. Запустите команду: `pip install torch==1.4 torchvision==0.5.0`

# Table of Contents

Это задание состоит из 5 частей. Вы изучите PyTorch на **трех разных уровнях абстракции**, что поможет вам лучше понять его и подготовить к финальному проекту. 

1. Часть I. Подготовка: мы будем использовать набор данных CIFAR-10. 
2. Часть II, Barebones PyTorch: **Уровень абстракции 1**, мы будем работать непосредственно с тензорами PyTorch самого низкого уровня. 
3. Часть III, API модуля PyTorch: **Уровень абстракции 2**, мы будем использовать `nn.Module` для определения произвольной архитектуры нейронной сети. 
4. Часть IV, PyTorch Sequential API: **Уровень абстракции 3**, мы будем использовать `nn.Sequential` для очень удобного определения линейной сети прямой связи. 
5. Часть V, открытое задание CIFAR-10: пожалуйста, создайте свою собственную сеть, чтобы получить максимально возможную точность на CIFAR-10. Вы можете экспериментировать с любым слоем, оптимизатором, гиперпараметрами или другими расширенными возможностями.

Вот таблица сравнения:

| API           | Flexibility | Convenience |
|---------------|-------------|-------------|
| Barebone      | High        | Low         |
| `nn.Module`     | High        | Medium      |
| `nn.Sequential` | Low         | High        |

# Part I. Preparation

Сначала мы загружаем набор данных CIFAR-10. В первый раз это может занять пару минут, но после этого файлы должны оставаться в кэше. 

В предыдущих частях задания нам приходилось писать собственный код для загрузки набора данных CIFAR-10, его предварительной обработки и итерирования по нему в мини-пакетах; PyTorch предоставляет нам удобные инструменты для автоматизации этого процесса.

In [6]:
import torch
#assert '.'.join(torch.__version__.split('.')[:2]) == '1.4'
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [7]:
NUM_TRAIN = 49000

# Пакет torchvision.transforms предоставляет инструменты для предварительной обработки данных 
# и выполнения аугментации данных; здесь мы настраиваем преобразование 
# для предварительной обработки данных путем вычитания среднего значения RGB 
# и деления на c.к.о. каждого значения RGB; 
# мы жестко закодировали среднее значение и с.к.о.

transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# Настраиваем объект Dataset для каждого разделения (train/val/test); 
# Наборы данных загружают обучающие примеры по одному, 
# поэтому мы оборачиваем каждый набор данных в DataLoader, 
# который перебирает набор данных и формирует мини-пакеты. 
# Мы делим обучающий набор CIFAR-10 на наборы train и val, 
# передавая объект Sampler в DataLoader, сообщая,
# как он должен выполнять выборку из базового набора данных.

# эти множества train,val,test могут пересечься?
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


У вас есть возможность **использовать GPU, установив флажок True ниже**. Для этого задания не обязательно использовать GPU. Обратите внимание, что если на вашем компьютере не включена CUDA, `torch.cuda.is_available()` вернет значение False, и этот блокнот вернется в режим ЦП. 

Глобальные переменные `dtype` и `device` будут управлять типами данных во всем этом задании.

## Colab Users

Если вы используете Colab, вам нужно вручную переключиться на устройство с графическим процессором. Вы можете сделать это, нажав `Runtime -> Change runtime type` и выбрав GPU под аппаратным ускорителем. Обратите внимание, что вам нужно повторно запускать ячейки сверху, так как ядро перезапускается при переключении сред выполнения.


In [8]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


# Part II. Barebones PyTorch

PyTorch поставляется с высокоуровневыми API, которые помогают нам удобно определять архитектуры моделей, которые мы рассмотрим во второй части этого руководства. В этом разделе мы начнем с базовых элементов PyTorch, чтобы лучше понять движок autograd. После этого упражнения вы оцените высокоуровневый API моделей. 

Мы начнем с простой полносвязной сети ReLU с двумя скрытыми слоями и без смещений для классификации CIFAR. Эта реализация вычисляет прямой проход, используя операции с тензорами PyTorch, и использует автоградиент PyTorch для вычисления градиентов. Важно, чтобы вы понимали каждую строчку, потому что после примера вы напишете более сложную версию.


Когда мы создаем тензор PyTorch с `requires_grad=True`, тогда операции с этим тензором будут не просто вычислять значения; они также будут строить вычислительный граф в фоновом режиме, что позволит нам легко выполнить обратное распространение по графу для вычисления градиентов некоторых тензоров по отношению к потерям в нисходящем направлении. Конкретно, если x является тензором с `x.requires_grad == True`, то после обратного распространения `x.grad` будет другим тензором, содержащим градиент x относительно скалярной loss в конце.

### PyTorch Tensors: Flatten Function


Тензор PyTorch концептуально похож на массив numpy: это n-мерная сетка чисел, и, как и numpy, PyTorch предоставляет множество функций для эффективной работы с тензорами. В качестве простого примера мы приводим ниже функцию  `flatten` , которая изменяет форму данных изображения для использования в полносвязной нейронной сети.

Напомним, что данные изображения обычно хранятся в тензоре формы N x C x H x W, где: 

* N — количество точек данных 
* C — количество каналов 
* H — высота промежуточной карты признаков в пикселях 
* W высота промежуточной карты признаков в пикселях

Это правильный способ представления данных, когда мы делаем что-то вроде 2D-свертки, для которой требуется пространственное понимание того, где промежуточные признаки относятся друг к другу. Однако, когда мы используем полностью связанные аффинные слои для обработки изображения, мы хотим, чтобы каждая точка данных была представлена одним вектором — больше нет смысла разделять различные каналы, строки и столбцы данных. Итак, мы используем операцию 'сгладить' (flatten), чтобы схлопнуть значения `C x H x W` для каждого представления в один длинный вектор. Приведенная ниже функция flatten сначала считывает значения N, C, H и W из заданного пакета данных, а затем возвращает «представление» (view) этих данных. «View» аналогичен методу «изменить форму» (reshape) numpy: он изменяет размеры x на N x ??, где ?? может быть любым (в данном случае это будет C x H x W, но нам не нужно указывать это явно).

In [17]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Barebones PyTorch: Two-Layer Network

Здесь мы определяем функцию `two_layer_fc`, которая выполняет прямой проход двухуровневой полносвязной сети ReLU для пакета данных изображения. После определения прямого прохода мы проверяем, что он не дает сбоев и выдает выходные данные правильной формы, пропуская нули через сеть.

Вам не нужно писать здесь какой-либо код, но важно, чтобы вы прочитали и поняли реализацию.

In [15]:
import torch.nn.functional as F  # полезные функции без сохранения состояния

def two_layer_fc(x, params):
    """
    A fully-connected neural networks; the architecture is:
    NN is fully connected -> ReLU -> fully connected layer.
    Note that this function only defines the forward pass; 
    PyTorch will take care of the backward pass for us.
    
    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.
    
    Inputs:
    - x: A PyTorch Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2] of PyTorch Tensors giving weights for the network;
      w1 has shape (D, H) and w2 has shape (H, C).
    
    Returns:
    - scores: A PyTorch Tensor of shape (N, C) giving classification scores for
      the input data x.
    """
    # first we flatten the image
    x = flatten(x)  # shape: [batch_size, C x H x W]
    
    w1, w2 = params
    
    # Прямой проход: вычисление предсказанного y с использованием операций с тензорами. 
    # Поскольку w1 и w2 имеют require_grad=True, 
    # операции с этими тензорами заставят PyTorch построить вычислительный граф, 
    # позволяющий автоматически вычислять градиенты.
    # Поскольку мы больше не реализуем обратный проход вручную, нам не нужно сохранять ссылки на промежуточные значения. 
    # вы также можете использовать `.clamp(min=0)`, что эквивалентно F.relu()

    x = F.relu(x.mm(w1))
    x = x.mm(w2)
    return x
    

def two_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 50), dtype=dtype)  # minibatch size 64, feature dimension 50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # you should see [64, 10]

two_layer_fc_test()

torch.Size([64, 10])


### Barebones PyTorch: Three-Layer ConvNet

Здесь вы завершите реализацию функции `three_layer_convnet`, которая будет выполнять прямой проход трехслойной сверточной сети. Как и выше, мы можем сразу протестировать нашу реализацию, пропустив нули по сети. Сеть должна иметь следующую архитектуру:

1. Сверточный слой (со смещением) с фильтрами `channel_1`, каждый из которых имеет форму `KW1 x KH1`, и zero-padding = 2 
2. Нелинейность ReLU 
3. Сверточный слой (со смещением) с фильтрами `channel_2`, каждый с формой `KW2 x KH2` и zero-padding = 1 
4. Нелинейность ReLU 
5. Полносвязный слой со смещением, дающий оценки для классов C.


Обратите внимание, что у нас нет **активации softmax** здесь после нашего полносвязного слоя: это потому, что потеря перекрестной энтропии PyTorch выполняет активацию softmax для вас, и объединение этого шага делает вычисления более эффективными.

**HINT**: Для сверток: http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d; обратите внимание на формы сверточных фильтров!

In [ ]:
def three_layer_convnet(x, params):
    """
    Performs the forward pass of a three-layer convolutional network with the
    architecture defined above.

    Inputs:
    - x: A PyTorch Tensor of shape (N, 3, H, W) giving a minibatch of images
    - params: A list of PyTorch Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: PyTorch Tensor of shape (channel_1, 3, KH1, KW1) giving weights
        for the first convolutional layer
      - conv_b1: PyTorch Tensor of shape (channel_1,) giving biases for the first
        convolutional layer
      - conv_w2: PyTorch Tensor of shape (channel_2, channel_1, KH2, KW2) giving
        weights for the second convolutional layer
      - conv_b2: PyTorch Tensor of shape (channel_2,) giving biases for the second
        convolutional layer
      - fc_w: PyTorch Tensor giving weights for the fully-connected layer. Can you
        figure out what the shape should be?
      - fc_b: PyTorch Tensor giving biases for the fully-connected layer. Can you
        figure out what the shape should be?
    
    Returns:
    - scores: PyTorch Tensor of shape (N, C) giving classification scores for x
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ################################################################################
    # TODO: Implement the forward pass for the three-layer ConvNet.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
  
    # (input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) → Tensor
    conv_1 = F.conv2d(x,conv_w1,conv_b1,padding = 2)
    conv_1 = F.relu(conv_1)

    conv_2 = F.conv2d(conv_1, conv_w2, conv_b2, padding = 1)
    conv_2 = F.relu(conv_2)

    conv_2 = conv_2.view(conv_2.shape[0], -1)
    scores = conv_2.mm(fc_w) + fc_b  # (N, channel_2*H2*W2 ) x (channel_2*H2*W2, 10)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################
    return scores

После определения прямого прохода ConvNet выше запустите следующую ячейку, чтобы проверить свою реализацию. 

Когда вы запускаете эту функцию, оценки должны иметь форму (64, 10).

In [ ]:
def three_layer_convnet_test():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]

    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel

    # you must calculate the shape of the tensor after two conv layers, before the fully-connected layer
    fc_w = torch.zeros((9 * 32 * 32, 10))
    fc_b = torch.zeros(10)

    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # you should see [64, 10]
three_layer_convnet_test()

torch.Size([64, 10])


### Barebones PyTorch: Initialization
Давайте напишем пару служебных методов для инициализации весовых матриц для наших моделей.

- `random_weight(shape)` инициализирует весовой тензор методом нормализации Кайминга. 
- `zero_weight(shape)` инициализирует весовой тензор всеми нулями. Полезно для создания экземпляров параметров смещения. 

Функция `random_weight` использует нормальный метод инициализации Kaiming, описанный в: 
He et al, *Delving Deep in Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [ ]:
def random_weight(shape):
    """
    Создавать случайные тензоры для весов; установка required_grad=True означает, 
    что мы хотим вычислить градиенты для этих тензоров во время обратного прохода. 
    Мы используем нормализацию Kaiming He: sqrt(2/fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn — стандартный генератор нормального распределения.
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU. 
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))

tensor([[-0.5858, -0.0562,  0.9618, -0.9284,  0.0144],
        [ 0.0816,  1.1724,  0.6152,  0.3826,  0.2840],
        [ 0.4542, -0.5675,  0.4305, -0.9298,  1.8583]], device='cuda:0',
       requires_grad=True)

### Barebones PyTorch: Check Accuracy
При обучении модели мы будем использовать следующую функцию, чтобы проверить точность нашей модели на обучающих или проверочных наборах. 

При проверке точности нам не нужно вычислять какие-либо градиенты; в результате нам не нужен PyTorch для построения вычислительного графика при вычислении оценок. Чтобы предотвратить построение графика, мы ограничиваем наши вычисления контекстным менеджером `torch.no_grad()`.

In [ ]:
def check_accuracy_part2(loader, model_fn, params):
    """
    Check the accuracy of a classification model.
    
    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model
    
    Returns: Nothing, but prints the accuracy of the model
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.int64)
            scores = model_fn(x, params)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### BareBones PyTorch: Training Loop
Теперь мы можем настроить базовый тренировочный цикл для обучения нашей сети. Мы будем обучать модель, используя стохастический градиентный спуск без импульса. Мы будем использовать `torch.functional.cross_entropy` для вычисления потерь; вы можете [прочитать об этом здесь](http://pytorch.org/docs/stable/nn.html#cross-entropy). 

Цикл обучения принимает на вход функцию нейронной сети, список инициализированных параметров (`[w1, w2]` в нашем примере) и скорость обучения.

In [ ]:
def train_part2(model_fn, params, learning_rate):
    """
    Train a model on CIFAR-10.
    
    Inputs:
    - model_fn: A Python function that performs the forward pass of the model.
      It should have the signature scores = model_fn(x, params) where x is a
      PyTorch Tensor of image data, params is a list of PyTorch Tensors giving
      model weights, and scores is a PyTorch Tensor of shape (N, C) giving
      scores for the elements in x.
    - params: List of PyTorch Tensors giving weights for the model
    - learning_rate: Python scalar giving the learning rate to use for SGD
    
    Returns: Nothing
    """
    for t, (x, y) in enumerate(loader_train):
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # Forward pass: compute scores and loss
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # Обратный проход: PyTorch определяет, какие тензоры в вычислительном 
        # графе имеют require_grad=True, и использует обратное распространение 
        # для вычисления градиента потерь по отношению к этим тензорам и сохраняет 
        # градиенты в атрибуте .grad каждого тензора.
        loss.backward()

        # Обновить параметры. Мы не хотим выполнять обратное 
        # распространение через обновления параметров, поэтому мы ограничиваем обновления контекстным менеджером torch.no_grad(),
        # чтобы предотвратить построение вычислительного графа.
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # Вручную обнулите градиенты после запуска обратного прохода
                w.grad.zero_()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params)
            print()

### BareBones PyTorch: Train a Two-Layer Network
Теперь мы готовы запустить обучающий цикл. Нам нужно явно выделить тензоры для полносвязных весов, `w1` и `w2`. 

Каждая мини-партия CIFAR содержит 64 примера, поэтому форма тензора — `[64, 3, 32, 32]`. 

После выравнивания форма `x` должна быть `[64, 3 * 32 * 32]`. Это будет размер первого измерения `w1`. Второе измерение `w1` — это размер скрытого слоя, который также будет первым измерением `w2`. 

Наконец, выход сети представляет собой 10-мерный вектор, представляющий распределение вероятностей по 10 классам. 

Вам не нужно настраивать какие-либо гиперпараметры, но вы должны увидеть точность выше 40% после обучения в течение одной эпохи.

In [ ]:
hidden_layer_size = 4000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, 10))

train_part2(two_layer_fc, [w1, w2], learning_rate)

Iteration 0, loss = 3.2562
Checking accuracy on the val set
Got 153 / 1000 correct (15.30%)

Iteration 100, loss = 2.3759
Checking accuracy on the val set
Got 375 / 1000 correct (37.50%)

Iteration 200, loss = 1.7924
Checking accuracy on the val set
Got 334 / 1000 correct (33.40%)

Iteration 300, loss = 1.7461
Checking accuracy on the val set
Got 402 / 1000 correct (40.20%)

Iteration 400, loss = 2.3272
Checking accuracy on the val set
Got 390 / 1000 correct (39.00%)

Iteration 500, loss = 2.0704
Checking accuracy on the val set
Got 411 / 1000 correct (41.10%)

Iteration 600, loss = 1.8602
Checking accuracy on the val set
Got 409 / 1000 correct (40.90%)

Iteration 700, loss = 1.6342
Checking accuracy on the val set
Got 415 / 1000 correct (41.50%)



### BareBones PyTorch: Training a ConvNet

Ниже вы должны использовать функции, определенные выше, для обучения трехслойной сверточной сети на CIFAR. Сеть должна иметь следующую архитектуру: 

1. Сверточный слой (со смещением) с 32 фильтрами 5x5 с заполнением нулями 2 
2. ReLU 
3. Сверточный слой (со смещением) с 16 фильтрами 3x3 с заполнением нулями 1 
4. ReLU 
5. Полносвязный слой (со смещением) для вычисления оценок для 10 классов 

Вы должны инициализировать свои весовые матрицы, используя функцию random_weight, определенную выше, и вы должны инициализировать свои векторы смещения, используя описанную выше функцию `zero_weight`. 

Вам не нужно настраивать какие-либо гиперпараметры, но если все работает правильно, вы должны достичь точности выше 42% после одной эпохи.

In [ ]:
learning_rate = 3e-3

channel_1 = 32
channel_2 = 16

conv_w1 = None
conv_b1 = None
conv_w2 = None
conv_b2 = None
fc_w = None
fc_b = None

################################################################################
# TODO: Initialize the parameters of a three-layer ConvNet.                    #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

conv_w1 = random_weight((channel_1, 3, 5, 5))
conv_b1 = zero_weight(channel_1)

conv_w2 = random_weight((channel_2, channel_1, 3, 3))
conv_b2 = zero_weight(channel_2)

fc_w = random_weight((channel_2*32*32, 10))
fc_b = zero_weight(10)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
train_part2(three_layer_convnet, params, learning_rate)

Iteration 0, loss = 2.9123
Checking accuracy on the val set
Got 105 / 1000 correct (10.50%)

Iteration 100, loss = 1.9749
Checking accuracy on the val set
Got 350 / 1000 correct (35.00%)

Iteration 200, loss = 1.7172
Checking accuracy on the val set
Got 383 / 1000 correct (38.30%)

Iteration 300, loss = 1.6311
Checking accuracy on the val set
Got 420 / 1000 correct (42.00%)

Iteration 400, loss = 1.5742
Checking accuracy on the val set
Got 428 / 1000 correct (42.80%)

Iteration 500, loss = 1.4530
Checking accuracy on the val set
Got 446 / 1000 correct (44.60%)

Iteration 600, loss = 1.4023
Checking accuracy on the val set
Got 462 / 1000 correct (46.20%)

Iteration 700, loss = 1.4064
Checking accuracy on the val set
Got 459 / 1000 correct (45.90%)



# Part III. PyTorch Module API

Barebone PyTorch требует, чтобы мы вручную отслеживали все тензоры параметров. Это хорошо для небольших сетей с несколькими тензорами, но было бы крайне неудобно и подвержено ошибкам отслеживать десятки или сотни тензоров в больших сетях. 

PyTorch предоставляет API `nn.Module`, с помощью которого вы можете определять произвольные сетевые архитектуры, отслеживая при этом все доступные для вас параметры. Во второй части мы сами реализовали SGD. PyTorch также предоставляет пакет `torch.optim`, который реализует все распространенные оптимизаторы, такие как RMSProp, Adagrad и Adam. Он даже поддерживает приближенные методы второго порядка, такие как L-BFGS! Вы можете обратиться к [doc](http://pytorch.org/docs/master/optim.html) для получения точных спецификаций каждого оптимизатора.


Чтобы использовать API модуля, выполните следующие действия: 

1. Подкласс `nn.Module`. Дайте вашему сетевому классу интуитивно понятное имя, например TwoLayerFC. 

2. В конструкторе `__init__()` определите все необходимые слои как атрибуты класса. Объекты слоя, такие как `nn.Linear` и `nn.Conv2d`, сами по себе являются подклассами `nn.Module` и содержат обучаемые параметры, поэтому вам не нужно создавать необработанные тензоры самостоятельно. `nn.Module` будет отслеживать эти внутренние параметры для вас. Обратитесь к [doc](http://pytorch.org/docs/master/nn.html), чтобы узнать больше о десятках встроенных слоев. **Внимание**: не забудьте сначала вызвать `super().__init__()`! 

3. В методе `forward()` определите *соединение* вашей сети. Вы должны использовать атрибуты, определенные в `__init__`, как вызовы функций, которые принимают тензор в качестве входных данных и выводят "преобразованный" тензор. *Не* создавайте новые слои с изучаемыми параметрами в `forward()`! Все они должны быть объявлены заранее в `__init__`. 

После того, как вы определите свой подкласс модуля, вы можете создать его экземпляр как объект и вызвать его точно так же, как функцию прямой прохода NN в части II.

### API модуля: двухуровневая сеть 
Вот конкретный пример двухуровневой полносвязной сети:

In [18]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # назначать объекты слоя атрибутам класса
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init пакет содержит удобные методы инициализации
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        # forward всегда определяет подключение
        x = flatten(x) # эту функцию сами определили выше в части 2.
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_TwoLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_TwoLayerFC()

torch.Size([64, 10])


### Module API: Three-Layer ConvNet

Теперь ваша очередь реализовать трехуровневую ConvNet, за которой следует полносвязный слой. Архитектура сети должна быть такой же, как и в части II: 

1. Сверточный слой с фильтрами 5x5 `channel_1` с заполнением нулями 2 
2. ReLU 
3. Сверточный слой с фильтрами 3x3 `channel_2` с заполнением нулями 1 
4. ReLU 
5. Полностью связанный слой с классами `num_classes` 

Вы должны инициализировать весовые матрицы модели, используя нормальный метод инициализации Кайминга.

 **СОВЕТ**: http://pytorch.org/docs/stable/nn.html#conv2d 
 
После реализации трехуровневой ConvNet функция `test_ThreeLayerConvNet` запустит вашу реализацию; он должен напечатать `(64, 10)` для формы выходных оценок.

In [ ]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        '''
        conv5x5 -> relu -> conv3x3 -> relu -> fc 
        '''
        self.conv1 = nn.Conv2d(in_channel, channel_1, (5,5), padding = (2,2), device = device ) # padding = "same" можно было так как у нас stride = 1
        self.conv2 = nn.Conv2d(channel_1, channel_2, (3,3), padding = "same", device = device )
        self.fc = nn.Linear(channel_2*32*32, 10, device = device)

        # инициализация весов слоев
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.fc.weight)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = x.to(device=device, dtype=dtype)
        conv_out = F.relu(self.conv2(F.relu(self.conv1(x)))) # для nn.ReLU: m = nn.ReLU   out = m(input)
        conv_out = conv_out.view(conv_out.shape[0], -1)
        scores = self.fc(conv_out)
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


def test_ThreeLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_ThreeLayerConvNet()

torch.Size([64, 10])


### Module API: Check Accuracy
Имея проверочный или тестовый набор, мы можем проверить точность классификации нейронной сети. 

Эта версия немного отличается от версии во второй части. Вы больше не передаете параметры вручную.

In [11]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # установить модель в режим оценки
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

### Module API: Training Loop
Мы также используем немного другой тренировочный цикл. Вместо самостоятельного обновления значений весов мы используем объект Optimizer из пакета `torch.optim`, который абстрагирует понятие алгоритма оптимизации и предоставляет реализации большинства алгоритмов, обычно используемых для оптимизации нейронных сетей.

In [10]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

        
            # Обнулите все градиенты для переменных, которые будет обновлять оптимизатор.
            optimizer.zero_grad()

            # Это обратный проход: вычислить градиент потери по отношению к каждому параметру модели.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

### Module API: Train a Two-Layer Network
Теперь мы готовы запустить обучающий цикл. В отличие от части II, мы больше не выделяем явно тензоры параметров. 

Просто передайте размер ввода, размер скрытого слоя и количество классов (т.е. размер вывода) конструктору `TwoLayerFC`.

Вам также необходимо определить оптимизатор, который отслеживает все обучаемые параметры внутри `TwoLayerFC`. Вам не нужно настраивать какие-либо гиперпараметры, но вы должны увидеть точность модели выше 40% после обучения в течение одной эпохи.

In [ ]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

Iteration 0, loss = 3.9317
Checking accuracy on validation set
Got 188 / 1000 correct (18.80)

Iteration 100, loss = 2.0891
Checking accuracy on validation set
Got 352 / 1000 correct (35.20)

Iteration 200, loss = 1.8417
Checking accuracy on validation set
Got 381 / 1000 correct (38.10)

Iteration 300, loss = 1.8980
Checking accuracy on validation set
Got 414 / 1000 correct (41.40)

Iteration 400, loss = 2.1986
Checking accuracy on validation set
Got 391 / 1000 correct (39.10)

Iteration 500, loss = 1.5180
Checking accuracy on validation set
Got 445 / 1000 correct (44.50)

Iteration 600, loss = 1.7388
Checking accuracy on validation set
Got 419 / 1000 correct (41.90)

Iteration 700, loss = 1.8891
Checking accuracy on validation set
Got 422 / 1000 correct (42.20)



### Module API: Train a Three-Layer ConvNet
Теперь вы должны использовать API модуля для обучения трехуровневой сети ConvNet на CIFAR. Это должно выглядеть очень похоже на обучение двухслойной сети! Вам не нужно настраивать какие-либо гиперпараметры, но вы должны достичь выше 45% после обучения в течение одной эпохи. 

Вы должны обучить модель, используя стохастический градиентный спуск без импульса.

In [ ]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None
################################################################################
# TODO: Instantiate your ThreeLayerConvNet model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = ThreeLayerConvNet(3, channel_1, channel_2, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 3.0579
Checking accuracy on validation set
Got 122 / 1000 correct (12.20)

Iteration 100, loss = 1.9414
Checking accuracy on validation set
Got 334 / 1000 correct (33.40)

Iteration 200, loss = 1.8284
Checking accuracy on validation set
Got 381 / 1000 correct (38.10)

Iteration 300, loss = 1.7455
Checking accuracy on validation set
Got 423 / 1000 correct (42.30)

Iteration 400, loss = 1.4992
Checking accuracy on validation set
Got 419 / 1000 correct (41.90)

Iteration 500, loss = 1.6114
Checking accuracy on validation set
Got 437 / 1000 correct (43.70)

Iteration 600, loss = 1.3020
Checking accuracy on validation set
Got 434 / 1000 correct (43.40)

Iteration 700, loss = 1.4912
Checking accuracy on validation set
Got 453 / 1000 correct (45.30)





Часть III представила API модуля PyTorch, который позволяет вам определять произвольные обучаемые слои и их связь. 

Для простых моделей, таких как стек слоев прямой связи, вам все равно нужно пройти 3 шага: подкласс `nn.Module`, назначить слои атрибутам класса в `__init__` и вызвать каждый слой один за другим в `forward()` . Есть ли более удобный способ? 

К счастью, PyTorch предоставляет модуль-контейнер под названием nn.Sequential, который объединяет вышеуказанные шаги в один. Он не такой гибкий, как `nn.Module`, потому что вы не можете указать более сложную топологию, чем стек с прямой связью, но этого достаточно для многих случаев использования. 

### Sequential API: двухуровневая сеть 

Давайте посмотрим, как переписать наш пример двухуровневой полносвязной сети с помощью `nn.Sequential` и обучить ее, используя описанный выше цикл обучения. Опять же, вам не нужно настраивать здесь какие-либо гиперпараметры, но вы должны достичь точности выше 40% после одной эпохи обучения.

In [16]:
# Нам нужно обернуть функцию `flatten` в модуль, чтобы сложить ее в nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# you can use Nesterov momentum in optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer)

Iteration 0, loss = 2.3374
Checking accuracy on validation set
Got 149 / 1000 correct (14.90)

Iteration 100, loss = 1.5450
Checking accuracy on validation set
Got 371 / 1000 correct (37.10)

Iteration 200, loss = 1.8890
Checking accuracy on validation set
Got 387 / 1000 correct (38.70)

Iteration 300, loss = 2.2270
Checking accuracy on validation set
Got 416 / 1000 correct (41.60)

Iteration 400, loss = 2.0906
Checking accuracy on validation set
Got 404 / 1000 correct (40.40)

Iteration 500, loss = 1.5994
Checking accuracy on validation set
Got 433 / 1000 correct (43.30)

Iteration 600, loss = 1.7978
Checking accuracy on validation set
Got 455 / 1000 correct (45.50)

Iteration 700, loss = 1.5488
Checking accuracy on validation set
Got 412 / 1000 correct (41.20)



### Sequential API: Three-Layer ConvNet

Здесь вы должны использовать `nn.Sequential` для определения и обучения трехслойной ConvNet с той же архитектурой, которую мы использовали в части III: 

1. Сверточный слой (со смещением) с 32 фильтрами 5x5, с нулевым заполнением 2 
2. ReLU 
3. Сверточный слой (со смещением) с 16 фильтрами 3x3 с заполнением нулями 1 
4. ReLU 
5. Полносвязный слой (со смещением) для вычисления оценок для 10 классов. 

Вы должны инициализировать свои весовые матрицы с помощью функции `random_weight`. определено выше, и вы должны инициализировать свои векторы смещения, используя функцию `zero_weight` выше. 

Вы должны оптимизировать свою модель, используя стохастический градиентный спуск с импульсом Нестерова 0.9. 

Опять же, вам не нужно настраивать какие-либо гиперпараметры, но вы должны увидеть точность выше 55% после одной эпохи обучения.

In [33]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

################################################################################
# TODO: Rewrite the 3-layer ConvNet with bias from Part III with the           #
# Sequential API.                                                              #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = nn.Sequential(
        nn.Conv2d(3, channel_1, 5, padding = 2),
        nn.ReLU(),
        nn.Conv2d(channel_1, channel_2, 3, padding = 1),
        nn.ReLU(),
        Flatten(),
        nn.Linear(channel_2*32*32,10)
        ).to(device)

'''
apply(fn): рекурсивно применяется fn к каждому подмодулю (возвращенному функцией .children()), 
а также к самому себе. Типичное использование включает инициализацию параметров модели.
'''
def weights_init(m):
    if isinstance(m, nn.Conv2d): # можно было и без условия на сверточный слой, но это для наглядности
        #nn.init.kaiming_normal_(m.weight.data)
        nn.init.kaiming_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data) 
    if isinstance(m, nn.Linear):
        #nn.init.kaiming_normal_(m.weight.data)
        nn.init.kaiming_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data)
    
#model.apply(weights_init) # инициализация весов. C инициализацией по умолчанию точность выше. WTF. 
# https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/linear.py#L44-L48 код инициализации по умолчанию. см reset_parameters(self)
# https://discuss.pytorch.org/t/whats-the-default-initialization-methods-for-layers/3157/19 # про не задокументированную инициализацию

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 2.3152
Checking accuracy on validation set
Got 150 / 1000 correct (15.00)

Iteration 100, loss = 1.4049
Checking accuracy on validation set
Got 444 / 1000 correct (44.40)

Iteration 200, loss = 1.3450
Checking accuracy on validation set
Got 492 / 1000 correct (49.20)

Iteration 300, loss = 1.2861
Checking accuracy on validation set
Got 511 / 1000 correct (51.10)

Iteration 400, loss = 1.1766
Checking accuracy on validation set
Got 515 / 1000 correct (51.50)

Iteration 500, loss = 1.2398
Checking accuracy on validation set
Got 522 / 1000 correct (52.20)

Iteration 600, loss = 1.2636
Checking accuracy on validation set
Got 570 / 1000 correct (57.00)

Iteration 700, loss = 1.1380
Checking accuracy on validation set
Got 573 / 1000 correct (57.30)



In [34]:
for i in model.children():
  print(i)

Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
ReLU()
Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU()
Flatten()
Linear(in_features=16384, out_features=10, bias=True)


# Part V. CIFAR-10 open-ended challenge

В этом разделе вы можете поэкспериментировать с любой архитектурой ConvNet на CIFAR-10, которая вам нравится. 

Теперь ваша задача — поэкспериментировать с архитектурами, гиперпараметрами, функциями потерь и оптимизаторами, чтобы обучить модель, которая достигает **по крайней мере 70%** точности на **валидационном** наборе CIFAR-10 в течение 10 эпох. Вы можете использовать функции check_accuracy и train сверху. Вы можете использовать либо `nn.Module`, либо `nn.Sequential` API.

Опишите, что вы сделали в конце этой тетради. 

Вот официальная документация API для каждого компонента. Одно замечание: то, что мы называем в классе «пространственной пакетной нормой», в PyTorch называется «BatchNorm2D». 

* Слои в пакете torch.nn: http://pytorch.org/docs/stable/nn.html 
* Активации: http://pytorch.org/docs/stable/nn.html#non-linear-activations 
* Функции потерь : http://pytorch.org/docs/stable/nn.html#loss-functions 
* Оптимизаторы: http://pytorch.org/docs/stable/optim.html


### Что вы можете попробовать: 
- **Размер фильтра**: Выше мы использовали 5x5; будут ли фильтры меньшего размера более эффективными? 
- **Количество фильтров**: выше мы использовали 32 фильтра. Больше или меньше делают лучше? 
- **Pooling против пошаговой свертки**: вы используете максимальное объединение или только пошаговые свертки? 
- **Пакетная нормализация**: попробуйте добавить пространственную пакетную нормализацию после сверточных слоев и ванильную пакетную нормализацию после аффинных слоев. Ваши сети обучаются быстрее? 
- **Сетевая архитектура**: приведенная выше сеть имеет три уровня обучаемых параметров. Можете ли вы добиться большего успеха с глубокой сетью? Хорошие архитектуры, которые можно попробовать, включают: 
- [conv-relu-pool]xN -> [affine]xM -> [softmax или SVM] 
- [conv-relu-conv-relu-pool]xN -> [affine]xM -> [ softmax или SVM] 
- [batchnorm-relu-conv]xN -> [affine]xM -> [softmax или SVM] 
- **Global Average Pooling**: вместо выравнивания (flattening) и последующего создания нескольких аффинных слоев выполняйте свертки до тех пор, пока ваше изображение становится маленьким (7x7 или около того), а затем выполняет операцию average pooling, чтобы получить изображение изображения 1x1 (1, 1, Filter#), которое затем преобразуется в вектор (Filter#). Это используется в [Google's Inception Network] (https://arxiv.org/abs/1512.00567) (см. Таблицу 1 для их архитектуры). 
- **Регуляризация**: добавьте регуляризацию веса l2 или, возможно, используйте Dropout.



### Tips for training
Для каждой сетевой архитектуры, которую вы пробуете, вы должны настроить скорость обучения и другие гиперпараметры. При этом следует помнить о нескольких важных вещах: 
- Если параметры работают хорошо, вы должны увидеть улучшение в течение нескольких сотен итераций 
- Помните подход от грубой к точной настройке гиперпараметров: начните с тестирования большого диапазона гиперпараметров всего за несколько итераций обучения, чтобы найти комбинации параметров, которые вообще работают. 
- После того, как вы нашли несколько наборов параметров, которые, по-видимому, работают, выполните более тщательный поиск по этим параметрам. Возможно, вам придется тренироваться для большего количества эпох. 
- Вам следует использовать проверочный набор для поиска гиперпараметров и сохранить тестовый набор для оценки вашей архитектуры по лучшим параметрам, выбранным проверочным набором.

### Going above and beyond (делая все возможное)
Если вы чувствуете себя авантюрным, есть много других особенностей, которые вы можете реализовать, чтобы попытаться улучшить свою производительность. Вы **не обязаны** реализовывать что-либо из этого, но не пропустите веселье, если у вас есть время! 

- Альтернативные оптимизаторы: вы можете попробовать Adam, Adagrad, RMSprop и т. д. 
- Альтернативные функции активации, такие как Leaky ReLU, PReLU, ELU или MaxOut. 
- Ансамбли моделей 
- Увеличение данных 
- Новые архитектуры 
  - [ResNets] (https://arxiv.org/abs/1512.03385), где входные данные из предыдущего слоя добавляются к выходным. 
  - [DenseNets] (https://arxiv.org/abs/1608.06993), где входные данные предыдущих слоев объединяются вместе. 
  - [В этом блоге есть подробный обзор] (https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

### Have fun and happy training! 

In [ ]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
model = None
optimizer = None

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

# You should get at least 70% accuracy
train_part34(model, optimizer, epochs=10)

## Describe what you did 

В ячейке ниже вы должны написать объяснение того, что вы сделали, любые дополнительные функции, которые вы реализовали, и/или любые графики, которые вы сделали в процессе обучения и оценки вашей сети.

TODO: Describe what you did

## Test set -- run this only once

Теперь, когда мы получили результат, которым мы довольны, мы тестируем нашу окончательную модель на тестовом наборе (которую вы должны сохранить в best_model). Подумайте, как это соотносится с точностью вашего проверочного набора.

In [ ]:
best_model = model
check_accuracy_part34(loader_test, best_model)